In [ ]:
# 将 llm_toy/src 加入 sys.path（稳健多候选）
import sys as _sys
from pathlib import Path as _Path
def _add_src_path():
    cands = [
        _Path.cwd()/'llm_toy'/'src',
        _Path.cwd()/'src',
        _Path.cwd().parent/'llm_toy'/'src',
        _Path.cwd().parent/'src',
    ]
    for base in list(_Path.cwd().parents)[:3]:
        cands += [base/'llm_toy'/'src', base/'src']
    for p in cands:
        if (p/'model.py').exists() and (p/'utils.py').exists():
            _sys.path.append(str(p.resolve()))
            print('已添加src路径:', p.resolve())
            return
    print('警告：未找到 llm_toy/src，请手动添加路径或调整工作目录。')
_add_src_path()


# Setup Troubleshooting Guide

本Notebook用于快速排查 PyTorch/CUDA/Jupyter 环境问题，并给出可行的替代方案（CPU模式）。

In [ ]:
import sys, platform, subprocess
import torch
print('=== System Information ===')
print('Python:', sys.version.split(' ')[0])
print('Platform:', platform.platform())
print('PyTorch:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
print('CUDA version:', torch.version.cuda)
print('cuDNN:', torch.backends.cudnn.version())


In [ ]:
# 检查 NVIDIA 驱动（若无命令则忽略）
try:
    out = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
    if out.returncode == 0:
        print('✅ NVIDIA driver is working')
        print('
'.join(out.stdout.split('
')[:5]))
    else:
        print('❌ nvidia-smi failed:')
        print(out.stderr)
except FileNotFoundError:
    print('❌ nvidia-smi not found - NVIDIA driver may not be installed')
except Exception as e:
    print('❌ Error running nvidia-smi:', e)


In [ ]:
# GPU 测试（可选）
if torch.cuda.is_available():
    print('使用GPU进行一次张量计算...')
    x = torch.randn(4096, 4096, device='cuda')
    y = x @ x.t()
    print('GPU计算成功:', tuple(y.shape))
else:
    print('CUDA不可用，跳过GPU测试。')


In [ ]:
# CPU 回退测试
x = torch.randn(1000, 1000)
y = x @ x.t()
print('CPU计算成功:', tuple(y.shape))


## 建议

- 驱动与CUDA匹配：优先使用与 PyTorch 发行版匹配的 CUDA（例如 cu118）。
- 若驱动不匹配：先在CPU模式学习，后续再切换GPU。
- 遇到导入报红：确保已安装依赖并重启内核，或将 llm_toy/src 标记为 Sources Root（PyCharm）。
